In [ ]:
import numpy as np
from mxnet import nd
from mxnet import image
import sys

sys.path.append('..')
import utils
from time import time
from mxnet import gluon
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from mxnet import init

classes = ['background', 'p1', 'p2', 'p3', 'p4',
           'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
           'p11', 'p12', 'p13', 'p14', 'p15',
           'p16', 'p17', 'p18', 'p19']
colormap = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0, 0, 128],
            [128, 0, 128], [0, 64, 128], [128, 128, 128], [64, 0, 0], [192, 0, 0],
            [64, 128, 0], [192, 128, 0], [64, 0, 128], [192, 0, 128],
            [64, 128, 128], [192, 128, 128], [0, 64, 0], [128, 64, 0],
            [0, 192, 0], [128, 192, 0]]

for _ in range(10):
    for expand_size in range(20, 85, 5):
        def image_crop(data, label, min_xy, max_xy, width=640, height=640):
            x0 = 504
            y0 = 697
            cropped_label = nd.zeros((data.shape[0], data.shape[1]))
            cropped_label[label[1] - expand_size:label[1] + expand_size,
            label[0] - expand_size:label[0] + expand_size] = 1
            return image.fixed_crop(data, x0, y0, width, height), cropped_label[y0:y0 + 640, x0:x0 + 640]


        data_root = '../data'
        image_root = data_root + '/CephalometricLandmark/RawImage'
        txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'


        def read_images(landmark_index, dataset_num=0):

            if dataset_num == 0:
                begin_index = 1
                end_index = 151
                image_path = image_root + '/TrainingData'
            elif dataset_num == 1:
                begin_index = 151
                end_index = 301
                image_path = image_root + '/Test1Data'
            else:
                begin_index = 301
                end_index = 401
                image_path = image_root + '/Test2Data'

            data, label = [None] * (end_index - begin_index), [None] * (end_index - begin_index)
            bounding_box = [[2400, 2400], [0, 0], [0, 0]]
            index = 0
            expand_size = 20
            for i in range(begin_index, end_index):
                image_filename = image_path + "/%03d.bmp" % i
                txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % i
                txt_filename2 = txt_root + '/400_junior' + "/%03d.txt" % i

                with open(txt_filename1, 'r') as f:
                    txts = f.read().split()
                x = int(txts[landmark_index].split(',')[0])
                y = int(txts[landmark_index].split(',')[1])

                bounding_box[0][0] = min(bounding_box[0][0], x)
                bounding_box[0][1] = min(bounding_box[0][1], y)
                bounding_box[1][0] = max(bounding_box[1][0], x)
                bounding_box[1][1] = max(bounding_box[1][1], y)
                bounding_box[2][0] = bounding_box[1][0] - bounding_box[0][0]
                bounding_box[2][1] = bounding_box[1][1] - bounding_box[0][1]

                image_tmp = image.imread(image_filename)
                data[index], label[index] = image_crop(image_tmp, [x, y], bounding_box[0], bounding_box[1])
                del image_tmp
                index += 1
            return data, label


        rgb_mean = nd.array([0.485, 0.456, 0.406])
        rgb_std = nd.array([0.229, 0.224, 0.225])


        def normalize_image(data):
            return (data.astype('float32') / 255 - rgb_mean) / rgb_std


        class VOCSegDataset(gluon.data.Dataset):

            def __init__(self, dataset_num, crop_size):
                self.crop_size = crop_size
                self.data, self.label = read_images(0, dataset_num=dataset_num)
                self.data[:] = [normalize_image(im) for im in self.data]

            def __getitem__(self, idx):
                return self.data[idx].transpose((2, 0, 1)), self.label[idx]

            def __len__(self):
                return len(self.data)


        input_shape = (640, 640)
        voc_train = VOCSegDataset(0, input_shape)
        voc_test1 = VOCSegDataset(1, input_shape)

        batch_size = 16
        train_data = gluon.data.DataLoader(
            voc_train, batch_size, shuffle=True, last_batch='discard')
        test_data = gluon.data.DataLoader(
            voc_test1, batch_size, last_batch='discard')

        conv = nn.Conv2D(10, kernel_size=4, padding=1, strides=2)
        conv_trans = nn.Conv2DTranspose(3, kernel_size=4, padding=1, strides=2)

        conv.initialize()
        conv_trans.initialize()

        pretrained_net = models.resnet18_v2(pretrained=True)

        net = nn.HybridSequential()
        for layer in pretrained_net.features[:-2]:
            net.add(layer)

        num_classes = len(classes)

        with net.name_scope():
            net.add(
                nn.Conv2D(2, kernel_size=1),
                nn.Conv2DTranspose(2, kernel_size=64, padding=16, strides=32)
            )


        def bilinear_kernel(in_channels, out_channels, kernel_size):
            factor = (kernel_size + 1) // 2
            if kernel_size % 2 == 1:
                center = factor - 1
            else:
                center = factor - 0.5
            og = np.ogrid[:kernel_size, :kernel_size]
            filt = (1 - abs(og[0] - center) / factor) * \
                   (1 - abs(og[1] - center) / factor)
            weight = np.zeros(
                (in_channels, out_channels, kernel_size, kernel_size),
                dtype='float32')
            weight[range(in_channels), range(out_channels), :, :] = filt
            return nd.array(weight)


        conv_trans = net[-1]
        conv_trans.initialize(init=init.Zero())
        net[-2].initialize(init=init.Xavier())

        x = nd.zeros((batch_size, 3, *input_shape))
        net(x)

        shape = conv_trans.weight.data().shape
        conv_trans.weight.set_data(bilinear_kernel(*shape[0:3]))

        for round_num in range(5):
            loss = gluon.loss.SoftmaxCrossEntropyLoss(axis=1)

            ctx = utils.try_all_gpus()
            net.collect_params().reset_ctx(ctx)

            trainer = gluon.Trainer(net.collect_params(),
                                    'sgd', {'learning_rate': .1, 'wd': 1e-3})

            utils.train(train_data, test_data, net, loss,
                        trainer, ctx, num_epochs=10)


            def predict(im):
                data = normalize_image(im)
                data = data.transpose((2, 0, 1)).expand_dims(axis=0)
                yhat = net(data.as_in_context(ctx[0]))
                pred = nd.argmax(yhat, axis=1)
                return pred.reshape((pred.shape[1], pred.shape[2]))


            def label2image(pred):
                x = pred.astype('int32').asnumpy()
                cm = nd.array(colormap).astype('uint8')
                return nd.array(cm[x, :])


            test_images, test_labels = read_images(0, 2)


            def evaluate_acc(result, label):
                if (len(np.where(result.asnumpy() > 0)[1]) == 0):
                    return False, False
                result_maxx = np.max(np.where(result.asnumpy() > 0)[1])
                result_minx = np.min(np.where(result.asnumpy() > 0)[1])
                result_maxy = np.max(np.where(result.asnumpy() > 0)[0])
                result_miny = np.min(np.where(result.asnumpy() > 0)[0])

                result_centerx = int((result_maxx + result_minx) / 2)
                result_centery = int((result_maxy + result_miny) / 2)

                result_avgx = np.average(np.where(result.asnumpy() > 0)[1])
                result_avgy = np.average(np.where(result.asnumpy() > 0)[0])

                label_maxx = np.max(np.where(label.asnumpy() > 0)[1])
                label_minx = np.min(np.where(label.asnumpy() > 0)[1])
                label_maxy = np.max(np.where(label.asnumpy() > 0)[0])
                label_miny = np.min(np.where(label.asnumpy() > 0)[0])

                label_centerx = int((label_maxx + label_minx) / 2)
                label_centery = int((label_maxy + label_miny) / 2)

                # lable_avgx = np.average(np.where(label.asnumpy() > 0)[1])
                # lable_avgy = np.average(np.where(label.asnumpy() > 0)[0])

                d1 = pow((result_centerx - label_centerx), 2) + pow((result_centery - label_centery), 2)
                d2 = pow((result_avgx - label_centerx), 2) + pow((result_avgy - label_centery), 2)

                return (d1 < 400, d2 < 400)


            n = 100
            imgs = []
            acc1 = 0
            acc2 = 0
            for i in range(n):
                x = test_images[i]
                result = predict(x)
                f1, f2 = evaluate_acc(result, test_labels[i])
                if f1:
                    acc1 += 1
                if f2:
                    acc2 += 1

            print("expand size : ", expand_size, " round : ", 10 + 10 * round_num, " acc1 : ", acc1 / n, " acc2 : ",acc2 / n)


expand size :  20  round :  10  acc1 :  0.02  acc2 :  0.02
expand size :  20  round :  20  acc1 :  0.5  acc2 :  0.5
expand size :  20  round :  30  acc1 :  0.73  acc2 :  0.74
expand size :  20  round :  40  acc1 :  0.77  acc2 :  0.78
expand size :  20  round :  50  acc1 :  0.83  acc2 :  0.83
expand size :  25  round :  10  acc1 :  0.74  acc2 :  0.74
expand size :  25  round :  20  acc1 :  0.84  acc2 :  0.84
expand size :  25  round :  30  acc1 :  0.84  acc2 :  0.85
expand size :  25  round :  40  acc1 :  0.84  acc2 :  0.85
expand size :  25  round :  50  acc1 :  0.84  acc2 :  0.85
expand size :  30  round :  10  acc1 :  0.8  acc2 :  0.83
expand size :  30  round :  20  acc1 :  0.86  acc2 :  0.87
expand size :  30  round :  30  acc1 :  0.89  acc2 :  0.9
expand size :  30  round :  40  acc1 :  0.9  acc2 :  0.9
expand size :  30  round :  50  acc1 :  0.91  acc2 :  0.92
expand size :  35  round :  10  acc1 :  0.77  acc2 :  0.77
expand size :  35  round :  20  acc1 :  0.88  acc2 :  0.89
exp

expand size :  30  round :  10  acc1 :  0.75  acc2 :  0.76
expand size :  30  round :  20  acc1 :  0.84  acc2 :  0.84
expand size :  30  round :  30  acc1 :  0.86  acc2 :  0.85
expand size :  30  round :  40  acc1 :  0.88  acc2 :  0.88
expand size :  30  round :  50  acc1 :  0.89  acc2 :  0.89
expand size :  35  round :  10  acc1 :  0.0  acc2 :  0.0
expand size :  35  round :  20  acc1 :  0.92  acc2 :  0.92
expand size :  35  round :  30  acc1 :  0.92  acc2 :  0.92
expand size :  35  round :  40  acc1 :  0.94  acc2 :  0.94
expand size :  35  round :  50  acc1 :  0.96  acc2 :  0.96
expand size :  40  round :  10  acc1 :  0.86  acc2 :  0.85
expand size :  40  round :  20  acc1 :  0.92  acc2 :  0.92
expand size :  40  round :  30  acc1 :  0.92  acc2 :  0.92
expand size :  40  round :  40  acc1 :  0.92  acc2 :  0.92
expand size :  40  round :  50  acc1 :  0.92  acc2 :  0.92
expand size :  45  round :  10  acc1 :  0.84  acc2 :  0.85
expand size :  45  round :  20  acc1 :  0.96  acc2 :  0.97

expand size :  40  round :  10  acc1 :  0.9  acc2 :  0.92
expand size :  40  round :  20  acc1 :  0.91  acc2 :  0.91
expand size :  40  round :  30  acc1 :  0.9  acc2 :  0.91
expand size :  40  round :  40  acc1 :  0.91  acc2 :  0.94
expand size :  40  round :  50  acc1 :  0.88  acc2 :  0.92
expand size :  45  round :  10  acc1 :  0.75  acc2 :  0.77
expand size :  45  round :  20  acc1 :  0.86  acc2 :  0.88
expand size :  45  round :  30  acc1 :  0.88  acc2 :  0.89
expand size :  45  round :  40  acc1 :  0.88  acc2 :  0.9
expand size :  45  round :  50  acc1 :  0.91  acc2 :  0.91
expand size :  50  round :  10  acc1 :  0.82  acc2 :  0.87
expand size :  50  round :  20  acc1 :  0.89  acc2 :  0.89
expand size :  50  round :  30  acc1 :  0.9  acc2 :  0.92
expand size :  50  round :  40  acc1 :  0.91  acc2 :  0.92
expand size :  50  round :  50  acc1 :  0.94  acc2 :  0.93
expand size :  55  round :  10  acc1 :  0.94  acc2 :  0.92
expand size :  55  round :  20  acc1 :  0.95  acc2 :  0.94
e

expand size :  50  round :  10  acc1 :  0.87  acc2 :  0.88
expand size :  50  round :  20  acc1 :  0.92  acc2 :  0.93
expand size :  50  round :  30  acc1 :  0.93  acc2 :  0.94
expand size :  50  round :  40  acc1 :  0.94  acc2 :  0.94
expand size :  50  round :  50  acc1 :  0.95  acc2 :  0.95
expand size :  55  round :  10  acc1 :  0.82  acc2 :  0.82
expand size :  55  round :  20  acc1 :  0.89  acc2 :  0.92
expand size :  55  round :  30  acc1 :  0.93  acc2 :  0.94
expand size :  55  round :  40  acc1 :  0.93  acc2 :  0.94
expand size :  55  round :  50  acc1 :  0.95  acc2 :  0.94
expand size :  60  round :  10  acc1 :  0.88  acc2 :  0.91
expand size :  60  round :  20  acc1 :  0.92  acc2 :  0.92
expand size :  60  round :  30  acc1 :  0.93  acc2 :  0.91
expand size :  60  round :  40  acc1 :  0.94  acc2 :  0.94
expand size :  60  round :  50  acc1 :  0.94  acc2 :  0.94
expand size :  65  round :  10  acc1 :  0.87  acc2 :  0.86
expand size :  65  round :  20  acc1 :  0.88  acc2 :  0.